** 📌 Step 1 — Install  Dependencies **

In [ ]:
!pip install gradio pandas sentence-transformers transformers faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 67.5 MB/s eta 0:00:00


**📌 Step 2 — Load Your Dataset**

In [ ]:
import pandas as pd

df = pd.read_csv('/content/Flipkart_Mobiles.csv')

df['combined_text'] = (
    df['Brand'].astype(str) + " | Model: " +
    df['Model'].astype(str) + " | Color: " +
    df['Color'].astype(str) + " | Memory: " +
    df['Memory'].astype(str) + " | Storage: " +
    df['Storage'].astype(str) + " | Rating: " +
    df['Rating'].astype(str) + " | Selling Price: " +
    df['Selling Price'].astype(str) + " | Original Price: " +
    df['Original Price'].astype(str)
)


In [ ]:
df.isnull().sum()

,0
Brand,0
Model,0
Color,0
Memory,0
Storage,0
Rating,0
Selling Price,0
Original Price,0
combined_text,0


In [ ]:
df = df.fillna("")

** 📌 Step 3 — Build Embeddings + FAISS Index **

In [ ]:
from sentence_transformers import SentenceTransformer
import faiss

model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

corpus = df['combined_text'].tolist()
corpus_embeddings = model.encode(corpus)

dimension = corpus_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(corpus_embeddings)


** 📌 Step 4 — Load Local LLM **

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
llm = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

** 📌 Step 5 — Chatbot Logic Function **

In [ ]:
def chatbot_response(query):
    query_vec = model.encode([query], convert_to_numpy=True)
    distances, indices = index.search(query_vec, 3)
    results = [corpus[i] for i in indices[0]]
    context = "\n".join(results)

    prompt = f"""
You are a helpful mobile expert chatbot. Use the data below to answer the question.

User Question:
{query}

Relevant Mobile Data:
{context}

Answer:
"""

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True)
    output = llm.generate(**inputs, max_length=350)
    return tokenizer.decode(output[0], skip_special_tokens=True)



In [ ]:
def chat_interface(message, chat_history):
    bot_reply = chatbot_response(message)
    chat_history.append((message, bot_reply))
    return "", chat_history


** 📌 Step 6 — Create a Full Chat UI with Gradio **

In [ ]:
import gradio as gr

def chat_interface(message, chat_history):
    bot_reply = chatbot_response(message)
    chat_history.append((message, bot_reply))
    return "", chat_history

with gr.Blocks() as demo:
    gr.Markdown("## 📱 Mobile Specification Chatbot")
    chatbot = gr.Chatbot()
    msg = gr.Textbox(label="Ask something about mobiles")
    msg.submit(chat_interface, [msg, chatbot], [msg, chatbot])

demo.launch()



/tmp/ipython-input-2334351171.py:10: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8225f44c708895a838.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
